In [13]:
#决策树 ID3算法
from math import log2
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

def entropy(labels):
    data_length=len(labels)
    result={}
    for each in labels:
        result[each]=result.get(each,0)+1
    ent=0
    for v in result.values():
        ent-=v/data_length*log2(v/data_length)
        
    return ent

#计算每一个属性的信息增益
def calcu_each_gain(column,update_data):
    total=len(column)
    #以特征列为基准来分组标签
    grouped=update_data.iloc[:,-1].groupby(by=column)
    
    #for g in list(grouped):
    #    con_ent+=len(g[1])/total*entropy(g[1])
    con_ent=sum ([len(g[1])/total*entropy(g[1]) for g in list(grouped)])
    return entropy(update_data.iloc[:,-1])-con_ent

#      找到最大增益作为分支准则
#返回（属性，增益）
def get_max_gain(temp_data):
    
    #计算每一个特征的增益，排序找出最大值
    #for feature_name in temp_data[:,:-1]取出的是属性名
    columns_entropy=[(feature,calcu_each_gain(temp_data[feature],temp_data))  for feature in temp_data.iloc[:,:-1].columns]
    columns_entropy=sorted(columns_entropy,key=lambda f: f[1], reverse=True)
    return columns_entropy[0]


#去掉数据中已存在的列属性内容
#并返回其每一个子节点对应的属性及样本数据
def drop_exist_feature(data,best_feature):
    attr=pd.unique(data[best_feature])
    new_data = [(nd, data[data[best_feature] == nd]) for nd in attr]
    new_data = [(n[0], n[1].drop([best_feature], axis=1)) for n in new_data]
    return new_data

#获取最多的label
def get_most_label(label_list):
    label_dict={}
    for i in label_list:
        label_dict[i]=label_dict.get(i,0)+1
    sorted_label=sorted(label_dict.items(),key=lambda t: t[1], reverse=True)
    #items返回的是一个元祖形式，（keys,values），key=lambda是将key函数应用于待排序元祖，再进行排序
    return sorted_label[0][0]

#创建决策树
#data_set为数据集
#column_count为属性特征集合

def create_tree(data_set,column_count):
    label_list=data_set.iloc[:,-1]#取出标签类
    if len(pd.unique(label_list))==1:
        #当前结点样本完全属于同一类，返回叶结点
        return label_list.values[0]
    if all([len(pd.unique(data_set[i]))==1 for i in data_set.iloc[:,:-1].columns]):
        #如果在特征上取值均相同则返回True
        #如果数据属性为空，则相当于all([]),也是返回True,即返回叶结点
        #某一属性下面只有一种特征，不可再分，返回叶结点，类标为种类最多的点
        return get_most_label(label_list)
    best_attr=get_max_gain(data_set)[0]
    #返回分支表现最好的属性名
    tree={best_attr:{}}
    #最好的属性下面的特征
    exist_attr=pd.unique(data_set[best_attr])
    #column_count为属性及特征集合
    if len(exist_attr)!=len(column_count[best_attr]):
        #即属性中某一特征的样本集为空
        no_exist_attr=set(column_count[best_attr])-set(exist_attr)
        for nea in no_exist_attr:
            tree[best_attr][nea]=get_most_label(label_list)
    for item in drop_exist_feature(data_set,best_attr):
        #item[0]为best_attr可能的取值
        tree[best_attr][item[0]]=create_tree(item[1],column_count)
    return tree


#用训练完成的DT预测数据
def classify(Tree, feature_names, test_data):
    classLabel=''
    #取出根结点
    root = list(Tree.keys())[0]
    #取出第一棵树
    firstDict = Tree[root]
    feature_index = feature_names.index(root)  #根节点的属性下标
    
    for key in firstDict.keys():   #根属性的取值,取哪个就走往哪颗子树,指定子树的走向
        if test_data[feature_index] == key:
            if type(firstDict[key]) == type({}):
                classLabel = classify(firstDict[key],feature_names,test_data)
            else:
                classLabel = firstDict[key]
    return classLabel


if __name__ == '__main__':
    
    my_data = pd.read_csv(r'E:\Repository\algorithms\xiguatest.txt',sep=',')
    column_count = dict([(ds, list(pd.unique(my_data[ds]))) for ds in my_data.iloc[:,:-1].columns])
    feature_names=list(my_data.iloc[:,:-1].columns)
    d_tree = create_tree(my_data, column_count)
    
    
    #print(d_tree)
    #print(feature_names.index('纹理'))
    #root=list(d_tree.keys())[0]
    #firstdict=d_tree[root]
    #create_plot(d_tree)       
    print(d_tree)
    test_data=pd.read_csv(r'E:\Repository\algorithms\xigua1.txt',sep=',')
    test=test_data.iloc[:,:].values
    for test_data in test:
        class_label=classify(d_tree, feature_names, test_data[0:-1])
        if class_label==test_data[-1]:
            print(True)
        else:
            print(False)
        
     

{'色泽': {'青绿': {'敲声': {'浊响': '是', '清脆': '否', '沉闷': '否'}}, '乌黑': {'根蒂': {'硬挺': '是', '蜷缩': '是', '稍蜷': {'纹理': {'模糊': '是', '稍糊': '是', '清晰': '否'}}}}, '浅白': '否'}}
False
False
False
False
True
True
False


In [48]:
import pandas as pd
test_data=pd.read_csv(r'E:\Repository\algorithms\xigua1.txt',sep=',')
a=test_data.iloc[:,:].values
for i in a:
    print(i[0:-1])


['青绿' '蜷缩' '沉闷' '稍糊' '稍凹' '硬滑']
['浅白' '蜷缩' '浊响' '清晰' '凹陷' '硬滑']
